## 1. FastAPI

The file name is `main.py`

```python
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/add")
def add_numbers(
    num1: int = Query(..., description="First number"),
    num2: int = Query(..., description="Second number")
):
    result = num1 + num2
    return {
        "num1": num1,
        "num2": num2,
        "sum": result
    }
```    

The run:

```bash
uvicorn main:app --reload
```

Now call:

```
curl "http://127.0.0.1:8000/add?num1=3&num2=7"
```

## 2. Query, Path, Body, Header, Cookie

FastAPI uses **special classes** (`Query`, `Path`, `Body`, `Header`, `Cookie`, …) to tell the framework **where to extract values from an HTTP request**.
The syntax is always the same:

```python
param_name: type = Source(default_value, extra_config...)
```

Where **Source** can be `Query`, `Path`, `Body`, etc.

---


### 2.1 Query
Extracts data from the **query string** (the `?foo=bar` part of the URL).


```python
num1: int = Query(..., description="First value")
```

---

#### 2.1.1 **`num1: int`**

This is a **type hint**.

* It says "`num1` must be an integer".
* FastAPI uses this to **validate and convert** the incoming query parameter.

---

#### 2.1.2 **`= Query(...)`**

`Query` is a helper function from FastAPI that lets you configure query parameters.

* `Query(...)`

  * The `...` (ellipsis) means **required**.
  * If you don’t pass `num1`, FastAPI will throw a validation error (422 Unprocessable Entity).
* You can also do:

  ```python
  num1: int = Query(None)   # optional, defaults to None
  num2: int = Query(10)  # optional, defaults to 10
  ```

So the ellipsis (`...`) = “no default value → must be provided”.

---

#### 2.1.3. **`description="First value"`**

This adds documentation to the parameter in the **OpenAPI schema**.

* When you open `/docs`, the Swagger UI will show “First value” next to the field.

---

###  2.2.`Path`

Extracts data from the **path parameters** (the `/items/{id}` part of the URL).

Example:

```python
from fastapi import Path

@app.get("/items/{item_id}")
def read_item(item_id: int = Path(..., description="The ID of the item", ge=1)):
    return {"item_id": item_id}
```

* `GET /items/123` → `{"item_id": 123}`
* `...` → required
* `ge=1` → validation (must be ≥ 1)

⚡ Difference vs `Query`:

* `Query` pulls from URL query string.
* `Path` pulls from the URL path itself.

---

## 2.3. Body

Extracts data from the **request body** (mainly used with `POST`, `PUT`, `PATCH`).
Usually combined with **Pydantic models** for validation.

Example:

```python
from fastapi import Body
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    price: float

@app.post("/items")
def create_item(item: Item = Body(..., description="Item details")):
    return {"name": item.name, "price": item.price}
```

Request:

```bash
curl -X POST "http://127.0.0.1:8000/items" \
     -H "Content-Type: application/json" \
     -d '{"name":"Laptop","price":999.99}'
```

Response:

```json
{"name":"Laptop","price":999.99}
```

⚡ Difference vs `Query`/`Path`:

* `Query`/`Path` handle small values (strings, ints, filters).
* `Body` handles structured JSON payloads.

---

**Quick Analogy**

Imagine a **restaurant order**:

* `Path` = the **table number** → fixed in the path (`/tables/7/order`).
* `Query` = the **extra instructions** you shout (`?extra_cheese=yes`).
* `Body` = the **actual order sheet** you hand to the waiter (JSON payload).

---

✅ TL;DR:

* **Query** → `/endpoint?foo=bar` (filters, search, options).
* **Path** → `/endpoint/{id}` (identifiers, resource locators).
* **Body** → request body, usually JSON (creation, updates, structured data).

---





## 3. Semantics of Endpoint

* **`GET`** → Used for *reading* data. Shouldn’t change the server state. Parameters go in the query string (`/get?foo1=...`).
* **`POST`** → Used for *creating or submitting* data. The body carries the payload (usually JSON). Most common when you’re sending data that may be large, sensitive, or complex.
* **`PUT`** → Used for *replacing/updating* an existing resource (idempotent: same call twice = same result).

In your case:

* If you just want a simple calculator (no state saved on server), `GET` is fine.
* If you expect **bigger/structured input** (e.g., arrays, JSON objects), then `POST` is more common.
* `PUT` would not be typical here, because you’re not updating a resource — just computing something.

Here’s the same example as `POST` with JSON input:

```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Numbers(BaseModel):
    foo1: int
    foo2: int

@app.post("/add")
def add_numbers(data: Numbers):
    result = data.foo1 + data.foo2
    return {"foo1": data.foo1, "foo2": data.foo2, "sum": result}
```

Call with `curl`:

```bash
curl -X POST "http://127.0.0.1:8000/add" \
     -H "Content-Type: application/json" \
     -d '{"foo1": 5, "foo2": 8}'
```

Response:

```json
{"foo1":5,"foo2":8,"sum":13}
```

---

So:

* **Testing, quick params** → `GET`
* **Real-world API, structured input** → `POST`




## 4. Add a production entrypoint (Gunicorn + Uvicorn workers)


### 4.1. Installation
- `Gunicorn` "Green Unicorn": is a Python Web Server Gateway Interface (WSGI) HTTP server.
- `Uvicorn` is an Asynchronous Server Gateway Interface web server (ASGI) implementation for Python.

ASGI and WSGI are protocols for communication between web servers and Python web applications. ASGI is newer, asynchronous, and more efficient for handling multiple requests simultaneously. WSGI is older, synchronous, and processes requests one at a time. The post explains their differences and provides example implementations of echo servers using both interfaces.

Ref: [1](https://www.reddit.com/r/Python/comments/1fr59e2/wtf_is_asgi_and_wsgi_in_python_apps_a_writeup/)


```bash
pip install "uvicorn[standard]" gunicorn
```

Run locally like production:

```bash
gunicorn -k uvicorn.workers.UvicornWorker -w 2 -b 0.0.0.0:8000 main:app
```

* `-k` option in Gunicorn** means **“worker class”** → what kind of worker processes Gunicorn should spawn to handle requests.

Gunicorn itself is just a **master process + worker manager**.
It doesn’t actually know how to talk HTTP → it delegates that job to **workers**.
Different `-k` values = different worker implementations.


- Each worker is a **Uvicorn process** that speaks **ASGI**.
- Required if you want **FastAPI’s async features + WebSockets** to work.
- Gunicorn = process manager; Uvicorn = actual HTTP/WebSocket server.

This is the recommended way to run FastAPI in production with Gunicorn.


* `-w` = workers (2–(CPU cores\*2)+1 is common starting point)
---



Add a simple health check:

```python
@app.get("/healthz")
def health():
    return {"status": "ok"}
```

### 4.2. Containerize (Docker)

**Dockerfile** (small + production-friendly):

```dockerfile
 
```

**requirements.txt**

```
fastapi
uvicorn[standard]
gunicorn
```

**.dockerignore**

```
__pycache__/
*.pyc
*.pyo
*.pyd
.venv/
venv/
.git/
.gitignore
.DS_Store
```

Build & run:

```bash
docker build -t my-fastapi .
docker run --rm -p 8000:8000 my-fastapi
```

### 4.3. Optional: docker-compose (with env vars)

**docker-compose.yml**

```yaml
version: "3.9"
services:
  api:
    build: .
    ports:
      - "8000:8000"
    environment:
      LOG_LEVEL: info
    restart: unless-stopped
```

### 4.4 Put a reverse proxy in front (TLS, gzip, caching)

Commonly **NGINX** or **Caddy** terminates HTTPS and forwards to the app container:

* NGINX → `proxy_pass http://api:8000;`
* Use Let’s Encrypt (Caddy makes this trivial) for TLS certificates.

### 4.5 Basic production checklist

* ✅ **Env vars/config**: don’t hardcode secrets; use `os.getenv`.
* ✅ **Validation & limits**: Pydantic models, `uvicorn`/NGINX body-size limits if needed.
* ✅ **CORS** (if a browser frontend hits your API):

  ```python
  from fastapi.middleware.cors import CORSMiddleware
  app.add_middleware(
      CORSMiddleware,
      allow_origins=["https://your-frontend.com"],
      allow_credentials=True,
      allow_methods=["GET","POST","PUT","DELETE","OPTIONS"],
      allow_headers=["*"],
  )
  ```
* ✅ **Logging**: structured logs (JSON) if you plan to ship logs; set Gunicorn log level.
* ✅ **Health/Readiness**: `/healthz`, `/readyz` (useful for Docker/K8s).
* ✅ **Testing**: `pytest`, `httpx`/`pytest-asyncio` for API tests.
* ✅ **DB migrations** (if any): Alembic for SQLAlchemy.
* ✅ **Observability**: metrics (Prometheus via `prometheus-fastapi-instrumentator`), tracing (OpenTelemetry), error reporting (Sentry).
* ✅ **Security**: rate limit at proxy, auth (OAuth2/JWT), keep dependencies updated.

## 6) Scaling options

* Single VM: Docker + NGINX is fine for small loads.
* Containers at scale: **Kubernetes** (readiness/liveness probes, HPA).
* Serverless: Cloud Run / Azure Container Apps / AWS App Runner (push your image and set min/max instances).

---

If you want, I can adapt the Dockerfile for:

* **multi-stage builds** (for compiled deps),
* **poetry** instead of `pip`,
* or a **compose** stack with NGINX in front.
